In [5]:
from keras.datasets import mnist
from sklearn.utils import shuffle
import math
import random
import numpy as np


import torch

if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)


Using TensorFlow backend.


gpu up


In [10]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()


In [19]:
threshold = 150  # for Binarize


def Binarize(X):
    X[X < threshold] = 0
    X[X >= threshold] = 1
    return X


train_X = Binarize(train_X)
test_X = Binarize(test_X)


def normalize(X):
    mx = np.max(X)
    return X/mx


In [16]:
def centroid(arr):
    moments = {'M00': arr.sum(axis=1).sum(),
               'M01': (np.arange(start=1, stop=arr.shape[1] + 1, step=1) * arr.sum(axis=0).reshape(1, -1)).sum(),
               'M10': (np.arange(start=1, stop=arr.shape[0] + 1, step=1) * arr.sum(axis=1).reshape(1, -1)).sum(),
               'M11': 0}

    x = moments['M10'] / moments['M00'] if moments['M00'] != 0 else 0
    y = moments['M01'] / moments['M00'] if moments['M00'] != 0 else 0
    return [x, y]


In [17]:
def imageWin(trainx, r, c):
    temp = []
    for i in range(0, int(trainx.shape[0]), r):
        for j in range(0, int(trainx.shape[1]), c):
            temp.append(centroid(trainx[i:i+r, j:j+c]))
    return np.array(temp)


def slicingArray(trainX, r, c):
    featureVector = []
    for trainx in trainX:
        featureVector.append(imageWin(trainx, r, c))
    return np.array(featureVector)


In [20]:
# Shuffling the dataset

# train_X, train_y = shuffle(train_X[:1500], train_y[:1500])
# test_X, test_y = shuffle(test_X[:100], test_y[:100])

train_X, train_y = train_X[:500], train_y[:500]
test_X, test_y = test_X[:100], test_y[:100]

arr = slicingArray(train_X, 4, 7)
trainFeatureVec = normalize(arr)
trainFeatureVec = trainFeatureVec.reshape([*arr.shape[:-2], -1])
trainFeatureVec = np.expand_dims(trainFeatureVec, axis=-1)


# print(trainFeatureVec.shape)

trainFeatureVec = trainFeatureVec.tolist()


arr = slicingArray(test_X, 4, 7)
testFeatureVec = normalize(arr)
testFeatureVec = testFeatureVec.reshape(*arr.shape[:-2], -1)
testFeatureVec = np.expand_dims(testFeatureVec, axis=-1)


train_y = (np.eye(10)[np.array(train_y)])
test_y = (np.eye(10)[np.array(test_y)])
train_y = np.expand_dims(train_y, axis=-1).tolist()
test_y = np.expand_dims(test_y, axis=-1).tolist()


In [21]:
# Some Fundamental Functions
def listXlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l1))]
    # print(len(l1),len(l1[0])," ",len(l2),len(l2[0]))
    for i in range(len(l1)):
        for j in range(len(l2[0])):
            for k in range(len(l2)):
                print(res[i][j])
                res[i][j] += l1[i][k] * l2[k][j]
    return res


def listMINUSlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l2))]
    for i in range(len(l2)):
        for j in range(len(l1[0])):
            res[i][j] = l1[i][j] - l2[i][j]
    return res


def listSquare(l: list):
    res = []
    for i in range(len(l)):
        res.append([l[i]**2])
    return res


def mean(k: list):
    return sum([i[0] for i in k]) / len(k)


def get_label(vector: list):
    mx = 0
    idx = 0
    for i in range(len(vector)):
        if(vector[i] > mx):
            mx = vector[i]
            idx = i
    return idx


def normalizeOut(vector: list):
    mx = vector[get_label(vector)]
    vector = [vector[i]/mx for i in range(len(vector))]
    return vector


def accuracy(target: list, predictions: list):
    trueValues = [get_label(target[i]) for i in range(len(target))]
    preds = [get_label(predictions[i]) for i in range(len(predictions))]
    cnt = 0
    for i in range(len(trueValues)):
        if trueValues[i] == preds[i]:
            cnt += 1

    return cnt / float(len(trueValues)) * 100.0


In [22]:
# Fully connected NN
class NN:
    def __init__(self, m: int, n: list, activation: str, e: int, lr: float, outShape: int):
        self.neurons = n                                                    # Neurons List to specify number of neurons in each layer
        self.neuronsListBefore = []                                         # Net values
        self.neuronsListAfter = []                                          # Neurons output values
        self.layers = m                                                     # Number of layers
        self.weights = dict([(x, [0]) for x in range(self.layers + 1)])     # Dictionary to save weights of each layer
        self.bias = dict([(x, [0]) for x in range(self.layers + 1)])        # Dictionary to save bias of each layer
        self.gradsWeights = []                                              # Dictionary to save weights gradients of each layer
        self.gradsBias = []                                                 # Dictionary to save bias gradients of each layer
        self.activationFn = activation                                      # String to specify layers' activation function
        self.learning_rate = lr                                         
        self.epochs = e
        self.outputShape = outShape                                         # Number of neurons in output layer

    # MSE
    def __costFunc(self, y, preds):
        return 0.5 * listSquare(listMINUSlist(y, preds)) /len(y)

    def __activation(self, x: float, type: str):
        if type == 'sigmoid':
            if x > 20:
                return 0.00000001
            elif x < -20:
                return 0.99999999
        res = {
            # 'sigmoid': 1 / (1 + math.exp(-x)),
            # 'tanh': math.tanh(x),
            # 'relu': max(x, 0),
            # 'linear': x,
            'leaky_relu': 0.1 * x if x <0 else x,
        }
        return res[type]

    def __derivatives(self, out: float, type: str):
        res = {
            # 'sigmoid': out * (1 - out),
            # 'tanh': 1 - (out ** 2),
            # 'relu': 0 if out < 0 else 1,
            # 'linear': 1,
            'leaky_relu': 0.1 if out < 0 else 1,
        }
        return res[type]

    def __initializeWeightsBias(self, input: list):
        self.weights[0] = [[random.random() for i in range(len(input))]
                           for j in range(self.neurons[0])]
        self.bias[0] = [random.random()
                        for i in range(self.neurons[0])]

        for i in range(1, self.layers):
            self.weights[i] = [[random.random() for i in range(
                self.neurons[i - 1])] for j in range(self.neurons[i])]
            self.bias[i] = [random.random()
                            for i in range(self.neurons[i])]

        self.weights[self.layers] = [[random.random() for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)]
        self.bias[self.layers] = [random.random()
                                  for i in range(self.outputShape)]

    def __zero_grads(self, input: list):
        self.gradsWeights = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsBias = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsWeights[0] = [[0 for i in range(len(input))]
                                for j in range(self.neurons[0])]
        self.gradsBias[0] = [0 for i in range(self.neurons[0])]

        for i in range(1, self.layers):
            self.gradsWeights[i] = [
                [0 for _ in range(self.neurons[i - 1])] for j in range(self.neurons[i])]
            self.gradsBias[i] = [0 for i in range(self.neurons[i])]

        self.gradsWeights[self.layers] = [[0 for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)]
        self.gradsBias[self.layers] = [0 for i in range(self.outputShape)]

    def __feedForward(self, input: list, layerIdx: int) -> list:
        out = []
        x = []
        # print(input,self.weights[layerIdx])
        # print(layerIdx)
        x = listXlist(self.weights[layerIdx], input)
        x = [[x[i][0] + self.bias[layerIdx][i]] for i in range(len(x))]
        self.neuronsListBefore.append(x)
        for i in range(len(x)):
            out.append([self.__activation(x[i][0], self.activationFn)])

        self.neuronsListAfter.append(out)
        return out

    def __propagate(self, x: list):
        self.neuronsListAfter.append(x)
        out = self.__feedForward(x, 0)
        for i in range(1, self.layers + 1):
            out = self.__feedForward(out, i)
        return out

    def __backPropagation(self, target: list, outputLayer: list):
        # print(len(outputLayer),outputLayer)
        dw = -1 * mean(listMINUSlist(target, outputLayer))
        deltas = dict([(x, [0]) for x in range(self.layers + 1)])
        deltas[self.layers] = np.array(
            [k[0] for k in listMINUSlist(target, outputLayer)])/-10

        for i in range(len(self.weights[self.layers])):
            deltas[self.layers][i] *= self.__derivatives(
                self.neuronsListBefore[self.layers][i][0], self.activationFn)

            self.gradsBias[self.layers][i] = deltas[self.layers][i]

            for j in range(len(self.weights[self.layers][0])):
                # print(self.neuronsListAfter[self.layers - 1])
                self.gradsWeights[self.layers][i][j] = deltas[self.layers][i] * \
                    self.neuronsListAfter[self.layers][j][0]

        # [sum([self.weights[j][i] * delta[layer + 1][j] for j in range(len(delta[layer]))]) for i in range(len(weights))]

        for layer in reversed(range(0, self.layers)):
            deltas[layer] = [sum([self.weights[layer + 1][j][i] * deltas[layer + 1][j]
                                 for j in range(len(deltas[layer]))]) for i in range(len(self.weights[layer + 1][0]))]
            for i in range(len(self.weights[layer])):
                deltas[layer][i] *= self.__derivatives(
                    self.neuronsListBefore[layer][i][0], self.activationFn)
                self.gradsBias[layer][i] = deltas[layer][i]
                # print(deltas)
                for j in range(len(self.weights[layer][0])):
                    # print(layer, i,j)
                    self.gradsWeights[layer][i][j] = deltas[layer][i] * \
                        self.neuronsListAfter[layer][j][0]

    def __updateWeightsBias(self):
        for layer in range(self.layers + 1):
            for i in range(len(self.weights[layer])):
                self.bias[layer][i] -= (self.learning_rate *
                                        self.gradsBias[layer][i])
                for j in range(len(self.weights[layer][0])):
                    self.weights[layer][i][j] -= (self.gradsWeights[layer]
                                                  [i][j] * self.learning_rate)

    def fit(self, trainX: list, trainY: list):
        self.__initializeWeightsBias(trainX[0])
        self.__zero_grads(trainX[0])
        for epoch in range(self.epochs):
            for i in range(len(trainX)):
                self.neuronsListBefore = []
                self.neuronsListAfter = []
                outputLayer = self.__propagate(trainX[i])
                self.__backPropagation(trainY[i], outputLayer)
                self.__updateWeightsBias()
                self.__zero_grads(trainX[i])

    def predict(self, testX: list):
        preds = []
        for i in range(len(testX)):
            preds.append(self.__propagate(testX[i]))
        return preds


In [23]:
# X = [[[i]] for i in range(50)]
# y = [[[d[0][0]*7+2]] for d in X]

# print(y[:5])

# nn = NN(2, [2, 2], 'linear', 1000, 0.0003, 1)
# nn.fit(X, y)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainFeatureVec = np.expand_dims(scaler.fit_transform(np.array(trainFeatureVec).squeeze(-1)),-1)



In [29]:
# nn.predict(X[:5])

nn = NN(2, [56,16], 'leaky_relu', 20, 0.000003, 10)
nn.fit(trainFeatureVec, train_y)

testpreds = nn.predict(trainFeatureVec)

0
0.0
0.0
-0.0941586976791341
-0.10064739968802025
-0.36028651592637306
-0.605410342744789
-0.6117249374357828
-0.6132265160436154
-0.8210341795974035
-0.9674029704295158
-0.2045159229757355
0.013149034403366677
0.3679529044349234
0.7981527389830267
1.1689596429873093
1.369367100055077
1.040147573862582
0.7114305243193992
0.6413815367048095
0.8862059792096616
0.7207845674477331
0.510828517458908
0.0688799670763256
-0.017733714608805115
-0.03506271486877231
-0.30367685869037403
-1.1719591070047874
-1.0222085504014842
-0.38580793719998285
-0.4751579575342164
-0.9318633857479901
-1.3657809798783793
-1.420308235758744
-1.56980171885419
-2.837599244894804
-3.7073481479046992
-3.576213047359218
-3.1970695690737254
-3.637536609628854
-3.7203167579787264
-2.3794081536901626
-0.8477893256374616
-0.38954673140528856
-0.38142230524133297
-0.4067907537972737
-0.40663925585333716
-0.596705898678024
-0.8310620328606382
0.5089943531741843
5.060289707847048
5.220072894201337
5.2186762424431645
5.12913

In [119]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=20,penalty='none')
lr.fit(np.array(trainFeatureVec).squeeze(-1),np.array(train_y).squeeze(-1).argmax(-1))

testpreds  = lr.predict(np.array(trainFeatureVec).squeeze(-1))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [123]:
sum([np.array(testpreds[i]).reshape(-1).argmax() ==np.array(train_y[i]).reshape(-1).argmax() for i in range(500)])/500

0.08

3

In [22]:
# nn.predict(X[:5])

nn.predict(testFeatureVec)

[[[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan], [nan]],
 [[nan], [nan], [nan], [nan], [nan], [nan], [nan], 